In [14]:
import torch
import os
import torch
import math
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from torch import Tensor
from joblib import dump, load
import pandas as pd 
import cv2
import gc

In [32]:
# -*- coding: utf-8 -*-
"""CapstoneProject_ProjectCode_NikhilKorlipara.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1l2PyW0zAGwOe4JMYFruFvACWYCTJ_7ZE
"""
class lazyload(Dataset):
    def __init__(self, path, isTrain=True, transform = None):
        self.transform = transform
        path = path + ('train/' if isTrain else 'test/')
        self.pathx = path + 'X/'
        self.pathy = path + 'Y/'
        self.data = os.listdir(self.pathx)
        self.isTrain = isTrain
    def __getitem__(self, idx):
        f = self.data[idx]
        img0 = cv2.imread(self.pathx + f + '/rgb/0.png')
        img1 = cv2.imread(self.pathx + f + '/rgb/1.png')
        img2 = cv2.imread(self.pathx + f + '/rgb/2.png')
        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        depth = np.load(self.pathx + f + '/depth.npy')
        field_id = pkl.load(open(self.pathx + f + '/field_id.pkl', 'rb'))
        if self.isTrain == False:
            return (img0, img1, img2, depth, torch.tensor(int(field_id)))
        y = np.load(self.pathy + f + '.npy')
        return (img0, img1, img2, depth, torch.tensor(int(field_id))), torch.tensor(y)
    def __len__(self):
        return len(self.data)

class norm_data:
    def conv_tensor_to_array(self, data, isTrain=True):
        len_data = len(data)
        if isTrain:
            (img0_, img1_, img2_, depth_, field_id_), y_ = data[0]
            img_shape, img1_shape, img2_shape, depth_shape, len_y = img0_.shape, img1_.shape, img2_.shape, depth_.shape, len(y_)
            y_ar = np.empty(shape=(len_data, len_y))
        else:
            (img0_, img1_, img2_, depth_, field_id_) = data[0]
            img_shape, img1_shape, img2_shape, depth_shape = img0_.shape, img1_.shape, img2_.shape, depth_.shape
            field_id_ar = np.empty(shape=(len_data, 1))
        img0_ar = np.empty(shape=(len_data, img_shape[0], img_shape[1], img_shape[2]))
        img1_ar = np.empty(shape=(len_data, img1_shape[0], img1_shape[1], img1_shape[2]))
        img2_ar = np.empty(shape=(len_data, img2_shape[0], img2_shape[1], img2_shape[2]))
        depth_ar = np.empty(shape=(len_data, depth_shape[0], depth_shape[1], depth_shape[2]))

        for inx, d in enumerate(data):
            # print(inx)
            if isTrain:
                (img0, img1, img2, depth, field_id), y = d
                y_ar[inx, :] = np.array(y)
                img0_ar[inx, :, :, :] = img0
                img1_ar[inx, :, :, :] = img1
                img2_ar[inx, :, :, :] = img2
                depth_ar[inx, :, :, :] = depth
            else:
                (img0, img1, img2, depth, field_id) = d
                field_id_ar[inx] = field_id.numpy()
                img0_ar[inx, :, :, :] = img0
                img1_ar[inx, :, :, :] = img1
                img2_ar[inx, :, :, :] = img2
                depth_ar[inx, :, :, :] = depth
        if isTrain:
            return img0_ar, img1_ar, img2_ar, depth_ar, y_ar
        return img0_ar, img1_ar, img2_ar, depth_ar, field_id_ar

    def norm_depth(self, depth):
        min_num = np.min(depth)
        max_num = np.max(depth)
        normalized_depth = (depth - min_num) / (max_num - min_num)
        return normalized_depth

    def norm_image(self, img):
        return (img/255.0)
      
    def add_depth_to_img(self, depth, img, which_img=0):
        new_img = np.empty(shape=(img.shape[0], img.shape[1], img.shape[2], img.shape[3]+1))
        depth0 = depth[:, which_img, :, :]
        for inx, _ in enumerate(img):
            depth_4d = np.expand_dims(depth0[inx], 2)
            new_img[inx] = np.concatenate((img[inx], depth_4d), axis=2)
        return new_img

    def data_reshaping(self, data):
        s = data.shape[0]
        c = data.shape[3]
        h = data.shape[1]
        w = data.shape[2]
        newdata = np.empty(shape=(s, c, h, w))

        for i in range(c):
            newdata[:, i, :, :] = data[:, :, :, i]
        return newdata

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
print(torch.cuda.is_available())
torch.cuda.empty_cache()

False


In [6]:
dp = norm_data()
data_train = lazyload('./Downloads/Capstone_Project/lazydata/', isTrain = True)
data_test = lazyload('./Downloads/Capstone_Project/lazydata/', isTrain = False)

In [7]:
gc.collect()
img0_array_test, img1_array_test, img2_array_test, depth_array_test, field_id_array = dp.conv_tensor_to_array(data=data_test, isTrain=False)
img0_array_train, img1_array_train, img2_array_train, depth_array_train, y_array = dp.conv_tensor_to_array(data=data_train, isTrain=True)

In [8]:
## print(type(depth_array_test), np.shape(depth_array_test))
depth_array_test = dp.norm_depth(depth_array_test)

img0_array_test = dp.norm_image(img0_array_test)
img0_array_test = dp.add_depth_to_img(depth_array_test, img0_array_test)
img0_array_test = dp.data_reshaping(img0_array_test)
gc.collect()

img1_array_test = dp.norm_image(img1_array_test)
img1_array_test = dp.add_depth_to_img(depth_array_test, img1_array_test)
img1_array_test = dp.data_reshaping(img1_array_test)
gc.collect()

img2_array_test = dp.norm_image(img2_array_test)
img2_array_test = dp.add_depth_to_img(depth_array_test, img2_array_test)
img2_array_test = dp.data_reshaping(img2_array_test)
gc.collect()

print('completed test reshaping')

depth_array_train = dp.norm_depth(depth_array_train)
print('completed normalized depth train')
gc.collect()

img0_array_train = dp.norm_image(img0_array_train)
img0_array_train = dp.add_depth_to_img(img=img0_array_train, depth=depth_array_train, which_img = 0)
img0_array_train = dp.data_reshaping(img0_array_train)
print('completed reshaping img0_array_train')
gc.collect()

img1_array_train = dp.norm_image(img1_array_train)
img1_array_train = dp.add_depth_to_img(img=img1_array_train, depth=depth_array_train, which_img = 1)
img1_array_train = dp.data_reshaping(img1_array_train)
print('completed reshaping img1_array_train')
gc.collect()

img2_array_train = dp.norm_image(img2_array_train)
img2_array_train = dp.add_depth_to_img(img=img2_array_train, depth=depth_array_train, which_img = 2)
img2_array_train = dp.data_reshaping(img2_array_train)
print('completed reshaping img2_array_train')
gc.collect()

completed test reshaping
completed normalized depth train
completed reshaping img0_array_train
completed reshaping new_img1_train
completed reshaping img2_array_train


0

In [9]:
testX_img0 = [img0_array_test, field_id_array]
testX_img1 = [img1_array_test, field_id_array]
testX_img2 = [img2_array_test, field_id_array]
train_img0 = [img0_array_train, y_array]
train_img1 = [img1_array_train, y_array]
train_img2 = [img2_array_train, y_array]

In [10]:
dump(testX_img0, 'preprocessed_test_img0.joblib')
dump(testX_img1, 'preprocessed_test_img1.joblib')
dump(testX_img2, 'preprocessed_test_img2.joblib')
dump(train_img0, 'preprocessed_train_data0.joblib')
dump(train_img1, 'preprocessed_train_data1.joblib')
dump(train_img2, 'preprocessed_train_data2.joblib')

['preprocessed_train_data2.joblib']